In [1]:
from pathlib import Path
import numpy as np
import matplotlib as plt
import k3d
import trimesh
import torch
import skimage

In [2]:
from exercise_3.model.threedepn import ThreeDEPNDecoder
from exercise_3.util.model import summarize_model

threedepn = ThreeDEPNDecoder()
print(summarize_model(threedepn))  # Expected: Rows 0-34 and TOTAL = 52455681

latent = torch.randn(16,256)
print(latent.shape)
predictions = threedepn(latent)

print('Output tensor shape: ', predictions.shape)

   | Name         | Type             | Params  
-----------------------------------------------------
0  | bottleneck   | Sequential       | 263168  
1  | bottleneck.0 | Linear           | 65792   
2  | bottleneck.1 | ReLU             | 0       
3  | bottleneck.2 | Linear           | 65792   
4  | bottleneck.3 | ReLU             | 0       
5  | bottleneck.4 | Linear           | 131584  
6  | bottleneck.5 | ReLU             | 0       
7  | decoder1     | Sequential       | 8389376 
8  | decoder1.0   | ConvTranspose3d  | 8388864 
9  | decoder1.1   | BatchNorm3d      | 512     
10 | decoder1.2   | ReLU             | 0       
11 | decoder2     | Sequential       | 2097536 
12 | decoder2.0   | ConvTranspose3d  | 2097280 
13 | decoder2.1   | BatchNorm3d      | 256     
14 | decoder2.2   | ReLU             | 0       
15 | decoder3     | Sequential       | 524480  
16 | decoder3.0   | ConvTranspose3d  | 524352  
17 | decoder3.1   | BatchNorm3d      | 128     
18 | decoder3.2   | ReLU          

In [2]:
from exercise_3.data.shapenet import ShapeNet

# Create a dataset with train split
train_dataset = ShapeNet('train')
val_dataset = ShapeNet('val')
overfit_dataset = ShapeNet('overfit')

# Get length, which is a call to __len__ function
print(f'Length of train set: {len(train_dataset)}')  # expected output: 153540
# Get length, which is a call to __len__ function
print(f'Length of val set: {len(val_dataset)}')  # expected output: 32304
# Get length, which is a call to __len__ function
print(f'Length of overfit set: {len(overfit_dataset)}')  # expected output: 64

Length of train set: 153540
Length of val set: 32304
Length of overfit set: 64


In [3]:
from exercise_3.util.visualization import visualize_mesh
from skimage.measure import marching_cubes

train_sample = train_dataset[1]
#print(f'Name: {train_sample["name"]}')  # expected output: 03001627/798a46965d9e0edfcea003eff0268278__3__-03001627/798a46965d9e0edfcea003eff0268278__0__
#print(f'Input SDF: {train_sample["input_sdf"].shape}')  # expected output: (2, 32, 32, 32)
print(f'Target DF: {train_sample["target_df"].shape}')  # expected output: (32, 32, 32)

input_mesh = marching_cubes(train_sample['target_df'], level=1)
#print(input_mesh)
visualize_mesh(input_mesh[0], input_mesh[1], flip_axes=True)

Target DF: (32, 32, 32)


Output()

In [5]:
from exercise_3.util.visualization import visualize_mesh
from skimage.measure import marching_cubes

train_sample = overfit_dataset[60]
#print(f'Name: {train_sample["name"]}')  # expected output: 03001627/798a46965d9e0edfcea003eff0268278__3__-03001627/798a46965d9e0edfcea003eff0268278__0__
#print(f'Input SDF: {train_sample["input_sdf"].shape}')  # expected output: (2, 32, 32, 32)
print(f'Target DF: {train_sample["target_df"].shape}')  # expected output: (32, 32, 32)

input_mesh = marching_cubes(train_sample['target_df'], level=1)
#print(input_mesh)
visualize_mesh(input_mesh[0], input_mesh[1], flip_axes=True)

Target DF: (32, 32, 32)


Output()

In [4]:
from exercise_3.training import train_3depn_vad
config = {
    'experiment_name': '3_1_3depn_overfitting',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': True,
    'batch_size': 32,
    'resume_ckpt': None,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 250,
    'print_every_n': 10,
    'validate_every_n': 250,
    'latent_code_length' : 256,
    'vad_free' : True,
    'kl_weight': 1

}
train_3depn_vad.main(config)

Using device: cuda:0
[004/00001] train_loss: 0.603385 kl_loss: 0.474221 normal_loss: 0.129164
[009/00001] train_loss: 0.461389 kl_loss: 0.422513 normal_loss: 0.038876
[014/00001] train_loss: 0.397052 kl_loss: 0.376322 normal_loss: 0.020729
[019/00001] train_loss: 0.348126 kl_loss: 0.335189 normal_loss: 0.012936
[024/00001] train_loss: 0.315695 kl_loss: 0.306336 normal_loss: 0.009360
[029/00001] train_loss: 0.297063 kl_loss: 0.289356 normal_loss: 0.007707
[034/00001] train_loss: 0.279856 kl_loss: 0.273371 normal_loss: 0.006485
[039/00001] train_loss: 0.263844 kl_loss: 0.258261 normal_loss: 0.005583
[044/00001] train_loss: 0.252087 kl_loss: 0.247060 normal_loss: 0.005027
[049/00001] train_loss: 0.244861 kl_loss: 0.240142 normal_loss: 0.004719
[054/00001] train_loss: 0.237838 kl_loss: 0.233395 normal_loss: 0.004443
[059/00001] train_loss: 0.231006 kl_loss: 0.226813 normal_loss: 0.004193
[064/00001] train_loss: 0.225806 kl_loss: 0.221803 normal_loss: 0.004003
[069/00001] train_loss: 0.2225

In [5]:
from exercise_3.util.visualization import visualize_mesh
from skimage.measure import marching_cubes
import torch.distributions as dist
model = ThreeDEPNDecoder()
model.load_state_dict(torch.load('exercise_3/runs/3_1_3depn_overfitting/' + "model_best.ckpt", map_location='cpu'))
latent_vectors = torch.load('exercise_3/runs/3_1_3depn_overfitting/latent_best.ckpt', map_location = 'cpu')
log_vars = torch.load('exercise_3/runs/3_1_3depn_overfitting/log_var_best.pt', map_location = 'cpu')
Dist = dist.Normal(latent_vectors[0], torch.exp(log_vars[0]))
x_vad = Dist.rsample().unsqueeze(0)
print(x_vad)
output_meshes = model(x_vad)
print(output_meshes.shape)
output_mesh = marching_cubes(output_meshes[0].detach().numpy(), level=1)
visualize_mesh(output_mesh[0], output_mesh[1], flip_axes=True)

tensor([[ 1.4780e+00,  1.8334e+00, -4.5796e-01, -1.5467e+00,  8.1558e-01,
         -3.4728e+00,  4.2831e-01, -2.6861e+00,  1.2894e+00, -3.4587e-01,
         -1.1912e+00,  7.7967e-01, -6.8007e-01, -1.7917e+00, -4.5616e-01,
          6.6990e-01,  1.7780e-01,  1.3631e-01, -3.6798e-01, -2.1832e-01,
         -1.6601e+00,  1.0758e+00, -2.6007e+00,  6.0293e-01, -7.6131e-01,
          1.8696e+00, -1.0097e+00, -4.5246e-01, -3.8259e-01,  5.3534e-02,
         -4.0509e-01, -1.7333e+00, -5.0495e-01, -5.3440e-01, -3.0962e+00,
          9.0052e-01, -4.4318e-01, -5.2586e-01, -1.3770e+00,  3.6186e-01,
          2.6785e+00, -1.8695e+00, -1.2665e+00, -6.0516e-01,  2.0079e+00,
          1.2437e+00, -5.9270e-01,  1.9756e+00,  1.5126e+00, -9.0876e-02,
          7.8496e-01, -1.5695e+00,  6.3331e-01,  1.6450e+00,  6.1957e-01,
          1.6435e+00, -1.5387e+00, -4.4077e-02,  8.5214e-01,  1.0975e+00,
         -5.5297e-01, -1.6584e+00,  1.2035e+00,  6.7995e-01, -1.3309e+00,
          2.0231e+00, -2.7912e+00, -8.

Output()

In [7]:
from exercise_3.training import train_3depn_vad
config = {
    'experiment_name': '3_1_3depn_generalization',
    'device': 'cuda:0',  # change this to cpu if you do not have a GPU
    'is_overfit': False,
    'batch_size': 32,
    'resume_ckpt': None,
    'learning_rate_model': 0.01,
    'learning_rate_code': 0.01,
    'learning_rate_log_var':0.01,
    'max_epochs': 100,
    'print_every_n': 100,
    'validate_every_n': 250,
    'latent_code_length' : 256,
    'vad_free' : True,
    'kl_weight': 10

}
train_3depn_vad.main(config)

Using device: cuda:0
[000/00099] train_loss: 5.043805 kl_loss: 0.500034 normal_loss: 0.043467
[000/00199] train_loss: 5.032662 kl_loss: 0.500567 normal_loss: 0.026997
[000/00299] train_loss: 5.031036 kl_loss: 0.500432 normal_loss: 0.026718
[000/00399] train_loss: 5.019615 kl_loss: 0.499309 normal_loss: 0.026521
[000/00499] train_loss: 5.027519 kl_loss: 0.500075 normal_loss: 0.026773
[000/00599] train_loss: 5.023463 kl_loss: 0.499668 normal_loss: 0.026786
[000/00699] train_loss: 5.026794 kl_loss: 0.500020 normal_loss: 0.026598
[000/00799] train_loss: 5.023679 kl_loss: 0.499737 normal_loss: 0.026304
[000/00899] train_loss: 5.022270 kl_loss: 0.499591 normal_loss: 0.026356
[000/00999] train_loss: 5.021141 kl_loss: 0.499511 normal_loss: 0.026030
[000/01099] train_loss: 5.029538 kl_loss: 0.500373 normal_loss: 0.025806
[000/01199] train_loss: 5.022107 kl_loss: 0.499605 normal_loss: 0.026054
[000/01299] train_loss: 5.022349 kl_loss: 0.499677 normal_loss: 0.025584
[000/01399] train_loss: 5.0322

[002/01701] train_loss: 2.579668 kl_loss: 0.255378 normal_loss: 0.025891
[002/01801] train_loss: 2.565586 kl_loss: 0.254031 normal_loss: 0.025271
[002/01901] train_loss: 2.571496 kl_loss: 0.254549 normal_loss: 0.026007
[002/02001] train_loss: 2.571892 kl_loss: 0.254605 normal_loss: 0.025843
[002/02101] train_loss: 2.568492 kl_loss: 0.254203 normal_loss: 0.026462
[002/02201] train_loss: 2.574309 kl_loss: 0.254866 normal_loss: 0.025646
[002/02301] train_loss: 2.571493 kl_loss: 0.254540 normal_loss: 0.026093
[002/02401] train_loss: 2.578455 kl_loss: 0.255224 normal_loss: 0.026217
[002/02501] train_loss: 2.572784 kl_loss: 0.254675 normal_loss: 0.026033
[002/02601] train_loss: 2.574578 kl_loss: 0.254817 normal_loss: 0.026409
[002/02701] train_loss: 2.568913 kl_loss: 0.254264 normal_loss: 0.026269
[002/02801] train_loss: 2.573739 kl_loss: 0.254772 normal_loss: 0.026017
[002/02901] train_loss: 2.576430 kl_loss: 0.255012 normal_loss: 0.026311


KeyboardInterrupt: 

In [20]:
from exercise_3.util.visualization import visualize_mesh
from skimage.measure import marching_cubes
import torch.distributions as dist
model = ThreeDEPNDecoder()
model.load_state_dict(torch.load('exercise_3/runs/3_1_3depn_generalization/' + "model_best.ckpt", map_location='cpu'))
latent_vectors = torch.load('exercise_3/runs/3_1_3depn_generalization/latent_best.pt', map_location = 'cpu')
log_vars = torch.load('exercise_3/runs/3_1_3depn_generalization/log_var_best.pt', map_location = 'cpu')
Dist = dist.Normal(latent_vectors[100], torch.exp(log_vars[100]))
x_vad = Dist.rsample().unsqueeze(0)
print(x_vad)
output_meshes = model(latent_vectors[1000].unsqueeze(0))
print(output_meshes.shape)
output_mesh = marching_cubes(output_meshes[0].detach().numpy(), level=1)
visualize_mesh(output_mesh[0], output_mesh[1], flip_axes=True)

tensor([[ 0.6584, -0.2321,  0.1266, -0.3753, -0.1055, -0.7185,  1.4486,  1.2286,
          1.1191, -1.2339,  0.8722, -1.1335, -0.9018, -1.0980, -0.6786,  0.9741,
          0.1818,  0.3132, -1.3102, -0.7462,  0.3455, -0.6517,  0.0623,  1.3077,
          0.9696,  2.8295,  2.1062, -2.5235,  0.4273, -1.4153, -0.6763,  0.7307,
         -0.2005,  1.4036,  0.3231, -0.4166, -2.8169, -0.8324, -0.0398, -0.8676,
          1.3576, -1.2660,  0.6033,  1.1655,  0.2042, -0.6970, -0.9532, -0.4612,
         -0.0757, -0.1920, -2.7722,  2.1989, -0.5961,  1.7815, -0.3403, -1.7410,
          0.4649,  2.2596, -1.7369, -1.3083,  0.5284,  0.0738,  1.0796, -1.8512,
         -1.2687, -1.0881,  0.8251, -0.4480, -2.3407,  1.7695,  0.1365,  0.3206,
         -0.3523, -0.4566,  1.2336,  0.6753, -1.0943, -0.9837,  0.6526,  1.8047,
         -0.0385, -0.8380,  0.9137, -1.4264, -0.9409, -0.6709,  0.3797, -0.7174,
          0.9260, -1.5662, -0.8791,  1.1298, -0.8859, -0.3710, -0.7087,  0.9470,
          1.2711,  2.3161, -

Output()